In [1]:
import numpy as np
from human_moveit_config.human_model import HumanModel
import transformations
import timeit
import rospy
import time
from scipy.spatial import cKDTree
from copy import deepcopy
import tf
import time

In [2]:
def create_database(link, base, joint_names, nb_points=1):
    X = []
    for i in range(nb_points):
        state = human.get_random_state()
        fk = human.forward_kinematic(state, links=link, base=base)
        joints = []
        for j in joint_names:
            joints.append(state.position[state.name.index(j)])
        # append quaternion in both hemispheres
        for i in range(2):
            x = deepcopy(joints)
            x += fk[link][0]
            x += ((-1)**i * np.array(fk[link][1]) / 40).tolist()
        X.append(x)
    names = deepcopy(joint_names)
    names += [link + '_x', link + '_y', link + '_z']
    names += [link + '_qx', link + '_qy', link + '_qz', link + '_qw']
    np.savez_compressed('/tmp/database_' + link, names=names, data=X)

In [3]:
rospy.init_node('test')

In [4]:
human = HumanModel()

In [5]:
links = ['shoulder_center', 'head', 'left_elbow', 'left_hand', 'right_elbow', 'right_hand']
bases = ['base', 'shoulder_center', 'shoulder_center', 'left_elbow', 'shoulder_center', 'right_elbow']
joint_by_links = {}
for l in links:
    joint_by_links[l] = human.joint_by_links[l]
for s in ['right', 'left']:
    joint_by_links[s + '_elbow'] = human.joint_by_links[s + '_shoulder'] + joint_by_links[s + '_elbow']

In [ ]:
nb_points = 10000
for i in range(len(links)):
    create_database(links[i], bases[i], joint_by_links[links[i]], nb_points=nb_points)

In [ ]:
test_links = ['shoulder_center', 'head']

In [ ]:
state = human.get_random_state()
fk_dict = {}
for t in test_links:
    fk_dict.update(human.forward_kinematic(state, links=t, base=bases[links.index(t)]))

In [ ]:
avg_time = 0
avg_prec = 0
for i in range(100):
    state = human.get_random_state()
    fk_test = human.forward_kinematic(state, links=links)
    start = time.time()
    ik = human.inverse_kinematic(fk_test)
    avg_time += (time.time() - start)
    ik_res = human.forward_kinematic(ik, links=links)
    for l in links:
        avg_prec += np.linalg.norm(np.array(fk_test[l][0]) - np.array(ik_res[l][0]))
        avg_prec += np.linalg.norm(np.array(fk_test[l][1]) - np.array(ik_res[l][1]))
avg_time /= 100
avg_prec /= (100 * len(links))

print 'time: ' + str(avg_time)
print 'precision: ' + str(avg_prec)

In [77]:
state = human.get_random_state()
fk_test = human.forward_kinematic(state, links=links)
ik = human.inverse_kinematic(fk_test)
test_link = links
ground_truth = human.forward_kinematic(state, links=test_link)
res_ik = human.forward_kinematic(ik, links=test_link)
for l in test_link:
    print l
    print ground_truth[l]
    print '-------------'
    print res_ik[l]
    print '##############'

shoulder_center
[[-0.20657746636609242, -0.17522952733381025, 0.10219529564600921], [0.31821587043255317, -0.5942002646097582, 0.1552925992674402, 0.7221834351155585]]
-------------
[[-0.20657774894649125, -0.17522899097549322, 0.102196728998494], [0.31821484510419407, -0.5941985967691388, 0.15529152117662726, 0.7221854909949031]]
##############
head
[[-0.33610416574944746, -0.3350122014842269, 0.2794845901200669], [0.21254337189060588, -0.14515443053274124, 0.13596927138664658, 0.95669632778676]]
-------------
[[-0.33610382125868044, -0.3350108641423042, 0.2794871134641955], [0.21254221464855408, -0.14515178340902865, 0.13596781963623558, 0.9566971928404695]]
##############
left_elbow
[[-0.4037388168726546, 0.1639357129798552, 0.36726864583852126], [-0.5706961273984278, -0.6809520984530736, 0.02855467964770495, 0.45803362328000535]]
-------------
[[-0.40373691784457494, 0.16393725028743747, 0.3672704211293285], [-0.5706972317135917, -0.6809500990768549, 0.028556139760444597, 0.4580351